# RAG from scratch

### References

* https://towardsdatascience.com/local-rag-from-scratch-3afc6d3dea08
* https://iamajithkumar.medium.com/how-to-use-chroma-to-build-your-first-similarity-search-5c054bfd5add


### Chunky

In [1]:
import re
import os
import uuid

from transformers import AutoTokenizer, AutoModel

c:\Users\HP\anaconda3\envs\nlp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import AutoModel, AutoTokenizer
import torch

model_name = "BAAI/bge-small-en-v1.5"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

tokenizer.save_pretrained("model/tokenizer")
model.save_pretrained("model/embedding")

In [3]:
def recursivelyChunkTillOptimumSize(
        chunks, 
        tokenizer, 
        chunk_size = 1024, 
        separator = None, 
        secondary_chunking_regex = None ):

    modifiedChunks = []
    refined = []
    current_chunk = ""
    # Split all words of size greater than chunk size into smaller chunks
    while len(chunks) > 0:
        chunk = chunks.pop(0)
        chunk = chunk.strip()

        if len(tokenizer.tokenize(chunk)) > chunk_size:
            sub_chunks = re.split(secondary_chunking_regex, current_chunk)
            chunks = sub_chunks + chunks
        else:
            modifiedChunks.append(chunk)

    # Join small chunks ot make big chunks
    current_chunk = ""
    for chunk in modifiedChunks:
        new_chunk = current_chunk + (separator if current_chunk else '') + chunk

        if len(tokenizer.tokenize(new_chunk)) <= chunk_size:
            current_chunk = new_chunk
        else:
            if current_chunk:
                refined.append(current_chunk)
            current_chunk = chunk

    if current_chunk:
        refined.append(current_chunk)

    return refined    
    
def chunk( 
        text, 
        tokenizer, 
        paragraph_separator = "\n\n", 
        chunk_size = 1024, 
        separator = " ", 
        secondary_chunking_regex = r'\S+?[\.,;!?]',
        chunk_overlap = 0 ):
    # Divide documents into paragraphs
    paragraphs = re.split(paragraph_separator, text)
    all_chunks = []

    for paragraph in paragraphs:
        words = paragraph.split(separator)
        chunks = recursivelyChunkTillOptimumSize(
            words, 
            tokenizer, 
            chunk_size = chunk_size, 
            separator = separator, 
            secondary_chunking_regex = secondary_chunking_regex 
        )
        all_chunks.extend(chunks)

    if chunk_overlap == 0:
        return all_chunks

    final_chunks = []
    for idx, chunk in enumerate(all_chunks):
        if idx == 0:
            final_chunks.append(chunk)
            continue
        # split the overlap between end of last chunk and start of next chunk
        overlap_chunk = all_chunks[idx - 1][overlap_chunk // 2: ] + all_chunks[idx][0: overlap_chunk // 2]
        final_chunks.extend( [overlap_chunk, chunk] )

    return final_chunks    

### Indexing

* Statisticals -> bag of words, sparse locations, etc
* Machine learned -> embedding

In [4]:
def compute_embeddings(chunk, tokenizer, model):
    inputs = tokenizer(chunk, return_tensors="pt", padding=True, truncation=True) 
    
    # Generate the embeddings 
    with torch.no_grad():    
        embeddings = model(**inputs).last_hidden_state.mean(dim=1).squeeze()

    return embeddings.tolist()

In [63]:
import chromadb
chroma_client = chromadb.PersistentClient(path="database4")
collection = chroma_client.create_collection(name="embeddings3", metadata={"hnsw:space": "cosine", "hnsw:M": 16, "hnsw:search_ef": 100 }, get_or_create = True)


In [64]:
data = {
    "MBR-2001": "Traditional sleigh bed crafted in rich walnut wood, featuring a curved headboard and footboard with intricate grain details. Queen size, includes a plush, supportive mattress. Produced by Heritage Bed Co. Dimensions: 65\"W x 85\"L x 50\"H.",
    "MBR-2002": "Art Deco-inspired vanity table in a polished ebony finish, featuring a tri-fold mirror and five drawers with crystal knobs. Includes a matching stool upholstered in silver velvet. Made by Luxe Interiors. Vanity dimensions: 48\"W x 20\"D x 30\"H, Stool dimensions: 22\"W x 16\"D x 18\"H.",
    "MBR-2003": "Set of sheer linen drapes in soft ivory, offering a delicate and airy touch to bedroom windows. Each panel measures 54\"W x 84\"L. Features hidden tabs for easy hanging. Manufactured by Tranquil Home Textiles.",

    "LVR-3001": "Convertible sofa bed upholstered in navy blue linen fabric, easily transitions from sofa to full-size sleeper. Perfect for guests or small living spaces. Features a sturdy wooden frame. Produced by SofaBed Solutions. Dimensions: 70\"W x 38\"D x 35\"H.",
    "LVR-3002": "Ornate Persian area rug in deep red and gold, hand-knotted from silk and wool. Adds a luxurious touch to any living room. Measures 8' x 10'. Manufactured by Ancient Weaves.",
    "LVR-3003": "Contemporary TV stand in matte black with tempered glass doors and chrome legs. Features integrated cable management and adjustable shelves. Accommodates up to 65-inch TVs. Made by Streamline Tech. Dimensions: 60\"W x 20\"D x 24\"H.",

    "OPT-4001": "Modular outdoor sofa set in espresso brown polyethylene wicker, includes three corner pieces and two armless chairs with water-resistant cushions in cream. Configurable to fit any patio space. Produced by Outdoor Living. Corner dimensions: 32\"W x 32\"D x 28\"H, Armless dimensions: 28\"W x 32\"D x 28\"H.",
    "OPT-4002": "Cantilever umbrella in sunflower yellow, featuring a 10-foot canopy and adjustable tilt for optimal shade. Constructed with a sturdy aluminum pole and fade-resistant fabric. Manufactured by Shade Masters. Dimensions: 120\"W x 120\"D x 96\"H.",
    "OPT-4003": "Rustic fire pit table made from faux stone, includes a natural gas hookup and a matching cover. Ideal for evening gatherings on the patio. Manufactured by Warmth Outdoor. Dimensions: 42\"W x 42\"D x 24\"H.",

    "ENT-5001": "Digital jukebox with touchscreen interface and built-in speakers, capable of streaming music and playing CDs. Retro design with modern technology, includes customizable LED lighting. Produced by RetroSound. Dimensions: 24\"W x 15\"D x 48\"H.",
    "ENT-5002": "Gaming console storage unit in sleek black, featuring designated compartments for systems, controllers, and games. Ventilated to prevent overheating. Manufactured by GameHub. Dimensions: 42\"W x 16\"D x 24\"H.",
    "ENT-5003": "Virtual reality gaming set by VR Innovations, includes headset, two motion controllers, and a charging station. Offers a comprehensive library of immersive games and experiences.",

    "KIT-6001": "Chef's rolling kitchen cart in stainless steel, features two shelves, a drawer, and towel bars. Portable and versatile, ideal for extra storage and workspace in the kitchen. Produced by KitchenAid. Dimensions: 30\"W x 18\"D x 36\"H.",
    "KIT-6002": "Contemporary pendant light cluster with three frosted glass shades, suspended from a polished nickel ceiling plate. Provides elegant, diffuse lighting over kitchen islands. Manufactured by Luminary Designs. Adjustable drop length up to 60\".",
    "KIT-6003": "Eight-piece ceramic dinnerware set in ocean blue, includes dinner plates, salad plates, bowls, and mugs. Dishwasher and microwave safe, adds a pop of color to any meal. Produced by Tabletop Trends.",

    "GBR-7001": "Twin-size daybed with trundle in brushed silver metal, ideal for guest rooms or small spaces. Includes two comfortable twin mattresses. Manufactured by Guestroom Gadgets. Bed dimensions: 79\"L x 42\"W x 34\"H.",
    "GBR-7002": "Wall art set featuring three abstract prints in blue and grey tones, framed in light wood. Each frame measures 24\"W x 36\"H. Adds a modern touch to guest bedrooms. Produced by Artistic Expressions.",
    "GBR-7003": "Set of two bedside lamps in brushed nickel with white fabric shades. Offers a soft, ambient light suitable for reading or relaxing in bed. Dimensions per lamp: 12\"W x 24\"H. Manufactured by Bright Nights.",

    "BMT-8001": "Industrial-style pool table with a slate top and black felt, includes cues, balls, and a rack. Perfect for entertaining and game nights in finished basements. Produced by Billiard Masters. Dimensions: 96\"L x 52\"W x 32\"H.",
    "BMT-8002": "Leather home theater recliner set in black, includes four connected seats with individual cup holders and storage compartments. Offers a luxurious movie-watching experience. Made by CinemaComfort. Dimensions per seat: 22\"W x 40\"D x 40\"H.",
    "BMT-8003": "Adjustable height pub table set with four stools, featuring a rustic wood finish and black metal frame. Ideal for casual dining or socializing in basements. Produced by Casual Home. Table dimensions: 36\"W x 36\"D x 42\"H, Stool dimensions: 15\"W x 15\"D x 30\"H."
}

In [65]:
import six

for sku, text in six.iteritems(data):
    chunks = chunk(text, tokenizer, chunk_overlap = 512)
    for created_chunk in chunks:
        embeddings = compute_embeddings(created_chunk, tokenizer, model)
        collection.add(ids = str(uuid.uuid1()), embeddings=embeddings, metadatas={ "sku": sku }, documents=created_chunk )

In [72]:
#question = "What does Virtual reality gaming include" 
#question = "color of cantilevel umbrella"
#question = "dimension of walnut wood sleigh bed"
#question = "color of twin size day-bed"
#question = "does the pool table have black felt"
#question = "what is the color of Twin-size daybed"
#question = "Frame size of wall art set"
#question = "what is the rolling kitchen cart made of"
question = "color of leather home theater recliner"

In [73]:
chunks = chunk(text, tokenizer, chunk_overlap = 512)
embeddings = []

for created_chunk in chunks:
    embedding = compute_embeddings(created_chunk, tokenizer, model)
    embeddings.append(embedding)

res = collection.query(
    query_embeddings=embeddings,
    n_results=100,
    include=['distances', 'documents', 'metadatas']
)

data = []
for i in range(len(res['ids'][0])):
    d = {}
    d['id'] = res['ids'][0][i]
    d['documents'] = res['documents'][0][i]
    d['distances'] = res['distances'][0][i]
    d['metadatas'] = res['metadatas'][0][i]

    data.append(d)
data = sorted(data, key = lambda x: x['distances'], reverse=True)[0: 4]

Number of requested results 100 is greater than number of elements in index 21, updating n_results = 21


In [74]:
data

[{'id': 'd97d8c86-2c6b-11ef-9232-70cf4972c938',
  'documents': 'Virtual reality gaming set by VR Innovations, includes headset, two motion controllers, and a charging station. Offers a comprehensive library of immersive games and experiences.',
  'distances': 0.5987853211979701,
  'metadatas': {'sku': 'ENT-5003'}},
 {'id': 'd96178ff-2c6b-11ef-ac0e-70cf4972c938',
  'documents': 'Cantilever umbrella in sunflower yellow, featuring a 10-foot canopy and adjustable tilt for optimal shade. Constructed with a sturdy aluminum pole and fade-resistant fabric. Manufactured by Shade Masters. Dimensions: 120"W x 120"D x 96"H.',
  'distances': 0.5148202416002798,
  'metadatas': {'sku': 'OPT-4002'}},
 {'id': 'd9704612-2c6b-11ef-8cfa-70cf4972c938',
  'documents': 'Digital jukebox with touchscreen interface and built-in speakers, capable of streaming music and playing CDs. Retro design with modern technology, includes customizable LED lighting. Produced by RetroSound. Dimensions: 24"W x 15"D x 48"H.',
 

In [75]:
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

qa_tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
qa_model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [76]:
def generateResponse(question, context, tokenizer, model):
    print(question)
    print(context)
    input_ids = tokenizer.encode(question, context)
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    sep_idx = tokens.index('[SEP]')
    token_type_ids = [0 for i in range(sep_idx+1)] + [1 for i in range(sep_idx+1,len(tokens))]

    out = model(torch.tensor([input_ids]), # The tokens representing our input text.
                token_type_ids=torch.tensor([token_type_ids]))

    start_logits,end_logits = out['start_logits'],out['end_logits']
    answer_start = torch.argmax(start_logits)
    answer_end = torch.argmax(end_logits)
    ans = ''.join(tokens[answer_start:answer_end])

    return ans


In [77]:
print(
    generateResponse(
        question,
        "".join([i['documents'] for i in data]),
        qa_tokenizer,
        qa_model
    )
)


color of leather home theater recliner
Virtual reality gaming set by VR Innovations, includes headset, two motion controllers, and a charging station. Offers a comprehensive library of immersive games and experiences.Cantilever umbrella in sunflower yellow, featuring a 10-foot canopy and adjustable tilt for optimal shade. Constructed with a sturdy aluminum pole and fade-resistant fabric. Manufactured by Shade Masters. Dimensions: 120"W x 120"D x 96"H.Digital jukebox with touchscreen interface and built-in speakers, capable of streaming music and playing CDs. Retro design with modern technology, includes customizable LED lighting. Produced by RetroSound. Dimensions: 24"W x 15"D x 48"H.Contemporary pendant light cluster with three frosted glass shades, suspended from a polished nickel ceiling plate. Provides elegant, diffuse lighting over kitchen islands. Manufactured by Luminary Designs. Adjustable drop length up to 60".

